In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz o cruzamento de deputados e doações - para saber deputados que podem estar nos embargos do Ibama ou doadores que estão na lista de embargos
# Dados em https://servicos.ibama.gov.br/ctf/publico/areasembargadas/ConsultaPublicaAreasEmbargadas.php

In [1]:
import pandas as pd

In [2]:
ibama = pd.read_excel('originais/embargos_ibama_1_fev_2022.xlsx', sheet_name='Planilha1')

In [3]:
ibama.columns

Index(['Nº TAD', 'SERIE TAD', 'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social',
       'CPF ou CNPJ', 'Localização do Imóvel', 'UF Infração',
       'Município Infração', 'Município Infrator', 'Bairro', 'Endereço',
       'Julgamento', 'Infração', 'Data de Inserção na Lista'],
      dtype='object')

In [4]:
ibama = ibama[pd.notnull(ibama['CPF ou CNPJ'])]

In [5]:
ibama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68162 entries, 0 to 76671
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Nº TAD                     68162 non-null  object        
 1   SERIE TAD                  62420 non-null  object        
 2   Área (ha)                  29827 non-null  float64       
 3   Nº A.I.                    62618 non-null  object        
 4   Nome ou Razão Social       68162 non-null  object        
 5   CPF ou CNPJ                68162 non-null  object        
 6   Localização do Imóvel      59512 non-null  object        
 7   UF Infração                68162 non-null  object        
 8   Município Infração         68162 non-null  object        
 9   Município Infrator         68071 non-null  object        
 10  Bairro                     65419 non-null  object        
 11  Endereço                   66552 non-null  object        
 12  Julg

In [6]:
ibama['CPF ou CNPJ']

0        27.433.203/0001-28
1        09.572.788/0001-85
2        03.532.877/0001-11
3        10.327.875/0001-56
4        13.660.639/0001-45
                ...        
76646    01.397.090/0001-69
76647        096.522.127-09
76649        226.416.291-00
76661        656.378.442-91
76671        105.025.358-22
Name: CPF ou CNPJ, Length: 68162, dtype: object

In [7]:
ibama.rename(columns = {'CPF ou CNPJ':'cpf_cnpj'},inplace = True)

In [8]:
def limpeza1(linha):
    documento = str(linha["cpf_cnpj"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

In [9]:
ibama["cpf_cnpj"] = ibama.apply(limpeza1, axis=1)

In [10]:
ibama["cpf_cnpj"]

0        27433203000128
1        09572788000185
2        03532877000111
3        10327875000156
4        13660639000145
              ...      
76646    01397090000169
76647       09652212709
76649       22641629100
76661       65637844291
76671       10502535822
Name: cpf_cnpj, Length: 68162, dtype: object

In [11]:
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cpf_cnpj"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado

In [12]:
ibama["cpf_cnpj"] = ibama.apply(compara1, axis=1)

In [13]:
def limpeza2(linha):
    documento = str(linha["cpf_cnpj"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

ibama['cnpj_raiz'] = ibama.apply(limpeza2, axis=1)

In [14]:
ibama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68162 entries, 0 to 76671
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Nº TAD                     68162 non-null  object        
 1   SERIE TAD                  62420 non-null  object        
 2   Área (ha)                  29827 non-null  float64       
 3   Nº A.I.                    62618 non-null  object        
 4   Nome ou Razão Social       68162 non-null  object        
 5   cpf_cnpj                   68162 non-null  object        
 6   Localização do Imóvel      59512 non-null  object        
 7   UF Infração                68162 non-null  object        
 8   Município Infração         68162 non-null  object        
 9   Município Infrator         68071 non-null  object        
 10  Bairro                     65419 non-null  object        
 11  Endereço                   66552 non-null  object        
 12  Julg

In [16]:
empresas_encontradas = ibama[(ibama['cpf_cnpj'] == '04856647040')]
empresas_encontradas

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz


In [18]:
empresas_encontradas = ibama[(ibama['cpf_cnpj'] == '30818125934')]
empresas_encontradas

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz


In [16]:
new = ibama["cnpj_raiz"].isin(["01463126", "02657326", "03167408", "04057706", "05904108", "07353227", "10242314", "12631778", "13134686", "14019929", "17331307", "21049762", "21129267", "22158240", "25555723", "26172122", "30202385", "32161939"])
ibama[new]                        

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz


In [15]:
ibama.to_excel('originais/copia_embargos_ibama_1_fev_2022.xlsx',sheet_name='Sheet1',index=False)

In [15]:
#ibama_menor = ibama[['cpf_cnpj', 'Nome ou Razão Social', 'cnpj_raiz']]

In [16]:
ibama_pf = ibama[(ibama['cnpj_raiz'] == 'documento_cpf')]
ibama_pj = ibama[(ibama['cnpj_raiz'] != 'documento_cpf')]

In [17]:
busca = ibama[(ibama['cpf_cnpj'] == '44444444444')]
busca

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz
48346,613647,E,NaN,NaN,Gerente Federal de Teste,44444444444,"Coordenada S 07º 53 53 43,40"" 54º 47 29,30 W, ...",PA,Altamira,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
49414,616490,E,NaN,NaN,Gerente Federal de Teste,44444444444,"Identificador 2014JAL050, Coordenadas Geográfi...",PA,Altamira,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
54346,638130,E,NaN,NaN,Gerente Federal de Teste,44444444444,Vicinal Curuá nas proximidades do Sítio 5 Irmã...,PA,Novo Progresso,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
59607,670275,E,284.03,NaN,Gerente Federal de Teste,44444444444,NaN,MT,Colniza,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2015-08-26,documento_cpf
60728,685907,E,99.30,NaN,Gerente Federal de Teste,44444444444,"BR 163 kM 1.000, Vicinal Pinheiro km 45.",PA,Novo Progresso,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2014-10-21,documento_cpf
61174,690170,E,NaN,NaN,Gerente Federal de Teste,44444444444,"ZONA RURAL DO MUNICIPIO DE COTRIGRAÇÚ/MT, NAS ...",MT,Cotriguaçu,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
61176,690173,E,NaN,NaN,Gerente Federal de Teste,44444444444,"PA JURUENA, ZONA RURAL DO MUNICIPIO DE JURUENA...",MT,Cotriguaçu,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
61789,702458,E,NaN,NaN,Gerente Federal de Teste,44444444444,"10°19'55,87"" S e 58°43'47,96"" W .",MT,Cotriguaçu,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-04-04,documento_cpf
64502,742632,E,NaN,NaN,Gerente Federal de Teste,44444444444,"Linha 02,localização RESEX JACIR PARANÁ Porto ...",RO,Porto Velho,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2018-05-10,documento_cpf
64623,744355,E,NaN,NaN,Gerente Federal de Teste,44444444444,NaN,AM,Lábrea,Brasília/DF,Asa Norte,Sede,pendente de julgamento,NaN,2017-09-11,documento_cpf


In [18]:
busca = ibama[(ibama['cpf_cnpj'] == '01522833404')]
busca

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz
16534,246378,C,4.66,680576,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Tamboril.,GO,Carmo do Rio Verde,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2014-06-26 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-11,documento_cpf
16535,246379,C,2.71,680577,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Fartura.,GO,Ceres,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2017-10-10 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-11,documento_cpf
16536,246380,C,2.05,680578,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Everest.,GO,Ceres,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2015-03-04 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-11,documento_cpf
16537,246381,C,3.44,680580,Paulo Fernando Cavalcanti de Morais,01522833404,"Fazenda ""A Bota"".",GO,Santa Isabel,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2015-03-04 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-11,documento_cpf
16538,246382,C,629.00,680581,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Boa Vista (Paulista).,GO,Carmo do Rio Verde,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2013-08-05 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-10,documento_cpf
35520,490683,C,9.20,483222,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Nova Zelândia.,GO,Santa Isabel,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2014-06-30 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-01-20,documento_cpf
35521,490684,C,1.11,483225,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Nova Zelândia.,GO,Santa Isabel,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2015-03-04 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-01-20,documento_cpf
35522,490685,C,2.45,483226,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Água Viva.,GO,Rialma,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2015-03-04 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-01-20,documento_cpf
35523,490686,C,1.89,483228,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Tamburil.,GO,Carmo do Rio Verde,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2014-09-10 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-11,documento_cpf
35524,490687,C,4.45,483229,Paulo Fernando Cavalcanti de Morais,01522833404,Fazenda Batatal.,GO,Carmo do Rio Verde,Carmo do Rio Verde/GO,Zona Suburbana,"Fazenda Boa Vista, Km 2,5, S/N",2014-07-02 00:00:00,"Destruir ou danificar florestas, cortar árvore...",2010-02-10,documento_cpf


In [15]:
busca = ibama[(ibama['cpf_cnpj'] == '62594370215')]
busca

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz
45279,586933,C,NaN,733155,Helder Zahluth Barbalho,62594370215,Estrada do Ariri.,PA,Ananindeua,Ananindeua/PA,Centro,"Av. Magalhães Barata, 115",pendente de julgamento,Causar poluição de qualquer natureza em níveis...,2013-01-08,documento_cpf


In [19]:
ibama_copia = ibama.copy()
ibama_copia.columns

Index(['Nº TAD', 'SERIE TAD', 'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social',
       'cpf_cnpj', 'Localização do Imóvel', 'UF Infração',
       'Município Infração', 'Município Infrator', 'Bairro', 'Endereço',
       'Julgamento', 'Infração', 'Data de Inserção na Lista', 'cnpj_raiz'],
      dtype='object')

In [20]:
ibama_copia.rename(columns = {'Município Infração':'municipio'},inplace = True)
ibama_copia.rename(columns = {'Infração':'tipo_infracao'},inplace = True)
ibama_copia.rename(columns = {'Data de Inserção na Lista':'data_fiscalizacao'},inplace = True)
ibama_copia.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
ibama_copia.rename(columns = {'Nome ou Razão Social':'entidade_nome'},inplace = True)
ibama_copia.rename(columns = {'UF Infração':'estado_id'},inplace = True)
ibama_copia.rename(columns = {'Área (ha)':'tamanho_area'},inplace = True)

In [21]:
ibama_copia = ibama_copia[['municipio', 'tipo_infracao', 'data_fiscalizacao', 'entidade_id', 'entidade_nome', 'estado_id', 'tamanho_area']]

In [23]:
ibama_copia.to_csv('resultados/embargos_ibama/lista_entidades_embargos_ibama.csv',index=False)

In [24]:
ibama.to_excel('resultados/embargos_ibama/lista_embargos_ibama_todas_colunas.xlsx',sheet_name='Sheet1',index=False)

### Doações

In [22]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
doacoes_2018 = pd.read_csv("resultados/doacoes/doacoes_geral_politicos_2018.csv", **kwargs)

In [23]:
def limpeza3(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_2018.apply(limpeza3, axis=1)

In [24]:
doacoes_2018_pf = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_2018_pj = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [25]:
doacoes_pj_ibama = pd.merge(doacoes_2018, ibama_pj.drop_duplicates('cnpj_raiz'), left_on='NR_CPF_CNPJ_DOADOR_raiz', right_on='cnpj_raiz', how = 'left')

In [26]:
conta = doacoes_pj_ibama[pd.notnull(doacoes_pj_ibama['cpf_cnpj'])]
conta.shape

(0, 32)

In [27]:
# Pessoa física

In [27]:
doacoes_pf_ibama = pd.merge(doacoes_2018, ibama_pf.drop_duplicates('cpf_cnpj'), left_on='NR_CPF_CNPJ_DOADOR', right_on='cpf_cnpj', how = 'left')

In [28]:
conta = doacoes_pf_ibama[pd.notnull(doacoes_pf_ibama['cpf_cnpj'])].copy()
conta.shape

(54, 32)

In [32]:
# Com todas colunas

In [153]:
conta.to_excel('resultados/embargos_ibama/doacoes_vindas_pessoas_com_embargos_ibama_2018.xlsx',sheet_name='Sheet1',index=False)

In [154]:
# Versão para BD

In [29]:
conta['doacoes_tipo'] = 'direta'

In [30]:
conta.columns

Index(['DT_RECEITA', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'NM_CANDIDATO',
       'DS_FONTE_RECEITA', 'DS_NATUREZA_RECEITA', 'NR_CPF_CNPJ_DOADOR',
       'NM_DOADOR_RFB', 'DS_ESFERA_PARTIDARIA_DOADOR', 'NM_PARTIDO_DOADOR',
       'SQ_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR', 'SG_PARTIDO_DOADOR',
       'DS_RECEITA', 'VR_RECEITA', 'NR_CPF_CNPJ_DOADOR_raiz', 'Nº TAD',
       'SERIE TAD', 'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social', 'cpf_cnpj',
       'Localização do Imóvel', 'UF Infração', 'Município Infração',
       'Município Infrator', 'Bairro', 'Endereço', 'Julgamento', 'Infração',
       'Data de Inserção na Lista', 'cnpj_raiz', 'doacoes_tipo'],
      dtype='object')

In [31]:
conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'DS_FONTE_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
conta.rename(columns = {'DT_RECEITA':'data'},inplace = True)
conta.rename(columns = {'NM_DOADOR_RFB':'entidade_nome'},inplace = True)
conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)

In [32]:
conta = conta[['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data']]

In [33]:
conta.columns

Index(['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo',
       'origem_do_dinheiro_no_tse', 'data'],
      dtype='object')

In [42]:
conta.shape

(54, 7)

In [34]:
conta

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
216,"1200,00",65595564253,50913735272,ANAILTON FRANCA PINHEIRO,direta,Outros Recursos,20/08/2018
1174,"2000,00",32770553291,43334490220,JOSE RONALDO DE MATOS DIAS,direta,Outros Recursos,27/08/2018
1411,"55000,00",79753566549,18609287734,LUCIANO MAFFRA DE VASCONCELLOS,direta,Outros Recursos,05/10/2018
1412,"70000,00",79753566549,18609287734,LUCIANO MAFFRA DE VASCONCELLOS,direta,Outros Recursos,18/09/2018
1533,"100000,00",38247186500,34814396600,LUTZ VIANA RODRIGUES JUNIOR,direta,Outros Recursos,04/10/2018
1534,"100000,00",38247186500,34814396600,LUTZ VIANA RODRIGUES JUNIOR,direta,Outros Recursos,04/09/2018
2319,"2524,54",04789113515,04789113515,JOSE ALVES ROCHA,direta,Outros Recursos,07/09/2018
2320,"9765,00",04789113515,04789113515,JOSE ALVES ROCHA,direta,Outros Recursos,07/09/2018
2999,"5000,00",49465589500,01754580904,ANTONIO CELSO SGANZERLA,direta,Outros Recursos,02/10/2018
4153,"275000,00",15791563809,66296676891,FERNANDO DE CASTRO MARQUES,direta,Outros Recursos,04/10/2018


In [43]:
conta.to_csv('resultados/embargos_ibama/doacoes_vindas_pessoas_com_embargos_ibama_2018.csv',index=False)

### Doações indiretas

In [35]:
doacoes_indiretas_2018 = pd.read_excel('resultados/doacoes/doacoes_todas_indiretas_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [36]:
doacoes_indiretas_2018.shape

(2372, 68)

In [37]:
doacoes_indiretas_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA',
       'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA', 'CD_NATUREZA_RECEITA',
       'DS_NATUREZA_RECEITA', 'CD_ESPECIE_RECEITA', 'DS_ESPECIE_RECEITA',
       'CD_CNAE_DOADOR', 'DS_CNAE_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR',
       'NM_DOADOR_RFB', 'CD_ESFERA_PARTIDARIA_DOADOR',
       'DS_ESFERA_PARTIDARIA_DOADOR', 'SG_UF_DOADOR', 'CD_MUNICIPIO_DOADOR',
       'NM_MUNICIPIO_DOADOR', 'SQ_CANDIDATO_DOADOR', 'NR_CANDIDATO_DOADOR',
       'CD_CARGO_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR

In [38]:
doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_indiretas_2018.apply(limpeza3, axis=1)

In [39]:
doacoes_indiretas_2018_pf = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_indiretas_2018_pj = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [40]:
doacoes_indiretas_2018_pf.shape

(0, 69)

In [41]:
doacoes_indiretas_2018_pj.shape

(2372, 69)

In [42]:
doacoes_pf_ibama_indireta = pd.merge(doacoes_indiretas_2018_pj, ibama_pf.drop_duplicates('cpf_cnpj'), left_on='cpf_doacoaindireta', right_on='cpf_cnpj', how = 'left')

In [43]:
conta = doacoes_pf_ibama_indireta[pd.notnull(doacoes_pf_ibama_indireta['cpf_cnpj'])].copy()
conta.shape

(39, 85)

In [58]:
conta.to_excel('resultados/embargos_ibama/doacoes_indiretas_vindas_pessoas_com_embargos_ibama_2018.xlsx',sheet_name='Sheet1',index=False)

In [45]:
conta['doacoes_tipo'] = 'indireta'

In [46]:
conta.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA',
       'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA', 'CD_NATUREZA_RECEITA',
       'DS_NATUREZA_RECEITA', 'CD_ESPECIE_RECEITA', 'DS_ESPECIE_RECEITA',
       'CD_CNAE_DOADOR', 'DS_CNAE_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR',
       'NM_DOADOR_RFB', 'CD_ESFERA_PARTIDARIA_DOADOR',
       'DS_ESFERA_PARTIDARIA_DOADOR', 'SG_UF_DOADOR', 'CD_MUNICIPIO_DOADOR',
       'NM_MUNICIPIO_DOADOR', 'SQ_CANDIDATO_DOADOR', 'NR_CANDIDATO_DOADOR',
       'CD_CARGO_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR

In [47]:
conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)
conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'DS_FONTE_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
conta.rename(columns = {'DT_RECEITA':'data'},inplace = True)
conta.rename(columns = {'SQ_CANDIDATO_DOADOR':'sequencialdoador_id'},inplace = True)
conta.rename(columns = {'NM_DOADOR_RFB':'entidade_nome'},inplace = True)

In [48]:
conta = conta[['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'sequencialdoador_id', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data']]

In [49]:
conta.columns

Index(['valor', 'politico_id', 'politico_id', 'entidade_id', 'entidade_nome',
       'sequencialdoador_id', 'doacoes_tipo', 'origem_do_dinheiro_no_tse',
       'data'],
      dtype='object')

In [59]:
conta.head()

,valor,politico_id,politico_id,entidade_id,entidade_nome,sequencialdoador_id,doacoes_tipo,origem_do_dinheiro_no_tse,data
144,"47000,00",07894090549,07894090549,04789113515,ELEICAO 2018 JOSE ALVES ROCHA DEPUTADO FEDERAL,50000607804,indireta,Fundo Especial,24/09/2018
145,"30000,00",07894090549,07894090549,04789113515,ELEICAO 2018 JOSE ALVES ROCHA DEPUTADO FEDERAL,50000607804,indireta,Fundo Especial,21/09/2018
146,"5000,00",07894090549,07894090549,04789113515,ELEICAO 2018 JOSE ALVES ROCHA DEPUTADO FEDERAL,50000607804,indireta,Fundo Especial,19/09/2018
147,"25000,00",07894090549,07894090549,04789113515,ELEICAO 2018 JOSE ALVES ROCHA DEPUTADO FEDERAL,50000607804,indireta,Fundo Especial,03/10/2018
457,"12500,00",25630512315,25630512315,03679004320,ELEICAO 2018 EUNICIO LOPES DE OLIVEIRA SENADOR,60000605116,indireta,Fundo Especial,03/10/2018


In [60]:
conta.shape

(39, 9)

In [61]:
conta.to_csv('resultados/embargos_ibama/doacoes_indiretas_vindas_pessoas_com_embargos_ibama_2018.csv',index=False)

### Doações originárias

In [62]:
doacoes_originarias_2018 = pd.read_excel('resultados/doacoes/doacoes_originarias_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [63]:
doacoes_originarias_2018.shape

(11060, 25)

In [64]:
doacoes_originarias_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'NR_CPF_CNPJ_DOADOR_ORIGINARIO', 'NM_DOADOR_ORIGINARIO',
       'NM_DOADOR_ORIGINARIO_RFB', 'TP_DOADOR_ORIGINARIO',
       'CD_CNAE_DOADOR_ORIGINARIO', 'DS_CNAE_DOADOR_ORIGINARIO', 'SQ_RECEITA',
       'DT_RECEITA', 'DS_RECEITA', 'VR_RECEITA', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO'],
      dtype='object')

In [65]:
def limpeza4(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR_ORIGINARIO"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] = doacoes_originarias_2018.apply(limpeza4, axis=1)

In [66]:
doacoes_originarias_2018_pf = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] == 'documento_cpf')]
doacoes_originarias_2018_pj = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] != 'documento_cpf')]

In [67]:
doacoes_originarias_2018_pf.shape

(11060, 26)

In [68]:
doacoes_originarias_2018_pj.shape

(0, 26)

In [71]:
doacoes_pf_ibama_originaria = pd.merge(doacoes_originarias_2018_pf, ibama_pf.drop_duplicates('cpf_cnpj'), left_on='NR_CPF_CNPJ_DOADOR_ORIGINARIO', right_on='cpf_cnpj', how = 'left')

In [72]:
conta = doacoes_pf_ibama_originaria[pd.notnull(doacoes_pf_ibama_originaria['cpf_cnpj'])].copy()
conta.shape

(3, 42)

In [90]:
conta.to_excel('resultados/embargos_ibama/doacoes_originarias_vindas_pessoas_com_embargos_ibama_2018.xlsx',sheet_name='Sheet1',index=False)

In [73]:
conta['doacoes_tipo'] = 'originaria'

In [74]:
conta.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'NR_CPF_CNPJ_DOADOR_ORIGINARIO', 'NM_DOADOR_ORIGINARIO',
       'NM_DOADOR_ORIGINARIO_RFB', 'TP_DOADOR_ORIGINARIO',
       'CD_CNAE_DOADOR_ORIGINARIO', 'DS_CNAE_DOADOR_ORIGINARIO', 'SQ_RECEITA',
       'DT_RECEITA', 'DS_RECEITA', 'VR_RECEITA', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz', 'Nº TAD',
       'SERIE TAD', 'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social', 'cpf_cnpj',
       'Localização do Imóvel', 'UF Infração', 'Município Infração',
       'Município Infrator', 'Bairro', 'Endereço', 'Julgamento', 'Infração',
       'Data de Inserção na Lista', 'cnpj_raiz', 'doacoes_tipo'],
      dtype='object')

In [75]:
conta.rename(columns = {'VR_RECEITA':'valor'},inplace = True)
conta.rename(columns = {'NR_CPF_CANDIDATO':'politico_id'},inplace = True)
conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'DS_RECEITA':'origem_do_dinheiro_no_tse'},inplace = True)
conta.rename(columns = {'DT_PRESTACAO_CONTAS':'data'},inplace = True)
conta.rename(columns = {'NM_DOADOR_ORIGINARIO_RFB':'entidade_nome'},inplace = True)

In [76]:
conta = conta[['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo', 'origem_do_dinheiro_no_tse', 'data']]

In [77]:
conta.columns

Index(['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo',
       'origem_do_dinheiro_no_tse', 'data'],
      dtype='object')

In [78]:
conta.head()

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
1914,"50,00",18836178200,64213838234,GIOVANNY SOUZA GUZZO,originaria,Recursos de Financiamento Coletivo,23/11/2018
8001,"1000,00",12519851813,14865544828,RENATA CHADE CATTINI MALUF,originaria,Recursos de Financiamento Coletivo,22/11/2018
9453,"500,00",10655365770,32823231900,NELSON RODRIGUEZ TEIXEIRA,originaria,Recursos de outros candidatos,05/11/2018


In [79]:
conta.to_csv('resultados/embargos_ibama/doacoes_originarias_vindas_pessoas_com_embargos_ibama_2018.csv',index=False)

### Candidatos 2018

In [20]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
politicos_2018 = pd.read_csv("resultados/politicos_2018_09abr.csv", **kwargs)

In [21]:
politicos_embargos_ibama = pd.merge(ibama_pf.drop_duplicates('cpf_cnpj'), politicos_2018, left_on='cpf_cnpj', right_on='politico_id', how = 'left')

In [22]:
conta = politicos_embargos_ibama[pd.notnull(politicos_embargos_ibama['politico_id'])].copy()
conta.shape

(6, 25)

In [101]:
conta

,Nº TAD,SERIE TAD,Área (ha),Nº A.I.,Nome ou Razão Social,cpf_cnpj,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,...,Data de Inserção na Lista,cnpj_raiz,nome,foto,nome_urna,politico_id,partido_eleicao,partido_atual,uri,sequencial_candidato
112,OC6733OJ,NaN,NaN,BUZ9TE0O,Charles Fernandes Silveira Santana,33938903520,próximo ao Assentamento,BA,Palmas de Monte Alto,Guanambi/BA,...,2021-12-06,documento_cpf,CHARLES FERNANDES SILVEIRA SANTANA,https://www.camara.leg.br/internet/deputado/ba...,Charles Fernandes,33938903520,PSD,PSD,https://dadosabertos.camara.leg.br/api/v2/depu...,50000607807
11560,246712,C,15.000,222035,Nelson Ned Previdente,01862797870,"Fazenda Netinho, coordenadas Lat. 17°44'82.3""S...",MT,Alto Taquari,Poloni/SP,...,2019-06-19,documento_cpf,NELSON NED PREVIDENTE,https://www.camara.leg.br/internet/deputado/ba...,Nelson Barbudo,01862797870,PSL,PSL,https://dadosabertos.camara.leg.br/api/v2/depu...,110000621666
14705,341746,C,32.670,212350,Jose Alves Rocha,04789113515,Fazenda São Lourenço.,BA,Coribe,Salvador/BA,...,2008-03-14,documento_cpf,JOSE ALVES ROCHA,https://www.camara.leg.br/internet/deputado/ba...,José Rocha,04789113515,PR,PL,https://dadosabertos.camara.leg.br/api/v2/depu...,50000607804
26379,522539,C,NaN,695855,Luciano Caldas Bivar,01818961415,FICA EMBARGADA TODA E QUALQUER ATIVIDADE DA MA...,PE,Jaboatão dos Guararapes,Jaboatão dos Guararapes/PE,...,2012-08-08,documento_cpf,LUCIANO CALDAS BIVAR,https://www.camara.leg.br/internet/deputado/ba...,Luciano Bivar,01818961415,PSL,PSL,https://dadosabertos.camara.leg.br/api/v2/depu...,170000617081
35352,634713,E,10.836,9095,Luiz Gonzaga Patriota,01960970453,"Fazenda Serra Santa, Petrolina/PE",PE,Petrolina,Brasília/DF,...,2017-04-04,documento_cpf,LUIZ GONZAGA PATRIOTA,https://www.camara.leg.br/internet/deputado/ba...,Gonzaga Patriota,01960970453,PSB,PSB,https://dadosabertos.camara.leg.br/api/v2/depu...,170000610284
42879,760092,E,13.677,9139076,Edio Vieira Lopes,77548981872,Fazenda São Carlos as margens da Br 174 - Zona...,RR,Mucajaí,Boa Vista/RR,...,2019-04-03,documento_cpf,EDIO VIEIRA LOPES,https://www.camara.leg.br/internet/deputado/ba...,Edio Lopes,77548981872,PR,PL,https://dadosabertos.camara.leg.br/api/v2/depu...,230000608387


In [168]:
conta.to_excel('resultados/embargos_ibama/deputados_2018_com_embargos_ibama.xlsx',sheet_name='Sheet1',index=False)

In [175]:
conta.rename(columns = {'cpf_cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'Município Infração':'municipio'},inplace = True)
conta.rename(columns = {'Infração':'tipo_infracao'},inplace = True)
conta.rename(columns = {'Data de Inserção na Lista':'data_insercao_lista'},inplace = True)
conta.rename(columns = {'UF Infração':'estado_id'},inplace = True)
conta.rename(columns = {'Área (ha)':'tamanho_area'},inplace = True)

In [176]:
conta = conta[['entidade_id', 'municipio', 'tipo_infracao', 'data_insercao_lista', 'estado_id', 'tamanho_area']]

In [178]:
conta.to_csv('resultados/embargos_ibama/deputados_2018_com_embargos_ibama.csv',index=False)